In [ ]:
!pip install -U langgraph langchain-community langchain-groq langchain-tavily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
!pip uninstall -y requests
!pip install requests==2.32.4
!pip install tavily

Found existing installation: requests 2.32.5
Uninstalling requests-2.32.5:
  Successfully uninstalled requests-2.32.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.29 requires requests<3,>=2.32.5, but you have requests 2.32.4 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily


In [ ]:
!pip install tavily-python

In [ ]:
from langchain.schema import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from tavily import TavilyClient
import os
from groq import Groq

In [ ]:
os.environ["TAVILY_API_KEY"] = "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"
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
client = Groq(api_key="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")

In [ ]:
user_context = {}
MAX_CLARIFICATIONS = 3

In [ ]:
def clarifier(user_question, user_context):
    """
    Ask up to MAX_CLARIFICATIONS questions for any user query.
    Returns either 'PROCEED' or a clarifying question string.
    """
    clarifier_prompt = f"""
You are a general-purpose question clarifier.
Your job is to check if the user's question is missing any important detail
that would help provide a precise and useful answer.

Rules:
- Ask **one clarifying question at a time**.
- Be natural and conversational.
- If the question is already clear enough, reply with 'PROCEED'.
- Consider any context already provided: {user_context}

User's question: "{user_question}"
Clarifier:
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": clarifier_prompt}],
        temperature=0.0
    )
    clarifier_text = response.choices[0].message.content.strip()
    return clarifier_text

In [ ]:
def deep_search_node(state):
    """
    Generate final answer from user's question after clarifications.
    Limit query length to avoid Tavily errors.
    """
    latest_question = state["messages"][-1].content

    # Combine essential context from user_context, not full user_question
    context_text = " ".join([f"{k}: {v}" for k, v in user_context.items()])
    full_query = f"{latest_question} {context_text}".strip()

    # Truncate to 400 chars to satisfy Tavily
    if len(full_query) > 400:
        full_query = full_query[-400:]  # take last 400 chars, usually latest info is more relevant

    # Perform Tavily search
    search_results = tavily.search(full_query, max_results=3)
    results = search_results.get("results", [])

    formatted_results = []
    for res in results:
        if isinstance(res, dict):
            url = res.get("url", "No URL")
            content = res.get("content", "")[:500]
            formatted_results.append(f"Source: {url}\nContent: {content}...")
        elif isinstance(res, str):
            formatted_results.append(f"Content: {res[:500]}...")

    if not formatted_results:
        formatted_results.append("No results found.")

    return {"messages": [AIMessage(content="\n\n".join(formatted_results))]}


In [ ]:
class ChatState(dict):
    messages: list

graph = StateGraph(ChatState)
graph.add_node("answer", deep_search_node)
graph.set_entry_point("answer")
graph.add_edge("answer", END)
app = graph.compile()

In [ ]:


# -------------------- Chat Loop --------------------
chat_history = []

while True:
    user_question = input("What's your question? (Type 'exit' to quit): ")
    if user_question.lower() == "exit":
        print("Goodbye!")
        break

    clarification_count = 0
    # Clarification loop
    while clarification_count < MAX_CLARIFICATIONS:
        clarifier_msg = clarifier(user_question, user_context)
        if clarifier_msg.strip().upper() == "PROCEED":
            break
        print(f"Assistant: {clarifier_msg}")
        user_input = input("Your answer: ")
        if user_input.lower() == "exit":
            break
        # Dynamically store context info
        user_context[clarifier_msg] = user_input
        user_question += " " + user_input
        clarification_count += 1

    # Add user question to chat history
    chat_history.append(HumanMessage(content=user_question))

    # Call LangGraph node for final answer
    for event in app.stream({"messages": chat_history}):
        for value in event.values():
            if value and isinstance(value.get("messages"), list):
                for msg in value["messages"]:
                    if isinstance(msg, AIMessage):
                        print("\nAssistant:", msg.content)


Assistant: To help me narrow down the search, what type of location or area are you currently in?
Assistant: So you're looking for a cafe in Bhubaneswar, Odisha. To give you the best possible recommendations, can you please tell me what part of Bhubaneswar you're currently in or would like to visit (like the Old Town or a specific area like Janpath or Patia)?
Assistant: So you're looking for a cafe in Bhubaneswar, Odisha. To give you the best possible recommendations, can you please tell me what part of Bhubaneswar you're currently in or would like to visit (like the Old Town or a specific area like Janpath or Patia) in Patia?

Assistant: Source: https://www.instagram.com/reel/DHlckjcP53R/?hl=en
Content: Photo by Nutmeg | Bhubaneswar on September 19, 2025. May be an image of....

Source: https://www.zomato.com/bhubaneswar/richards-kitchen-coffee-bar-patia-bhubaneshwar
Content: Richard's Kitchen & Coffee Bar. Cafe, Chinese, Fast Food, Seafood, Sichuan. Plot A/2, 4th Floor, KIIT Square, 